# 学習、テスト

学習とテストを分ける。 8 : 2  
学習をcross validationし、ハイパーパラメーターを決める。  
そのハイパーパラメーターを使って、学習データすべてで学習する。
テストする。

# hold out

In [ ]:
# 1回分割
from sklearn.model_selection import ShuffleSplit
ss = ShuffleSplit(n_splits=1,      # 分割を1個生成
                  train_size=0.8,  # 学習は8割
                  test_size =0.2,  # テストは2割
                  random_state=0)  # 乱数種（再現用）
train_index, test_index = next(ss.split(X)) 
X_train, X_test = X[train_index], X[test_index] # 学習データ，テストデータ
y_train, y_test = y[train_index], y[test_index] # 学習データのラベル，テストデータのラベル

# 複数回分割
ss = ShuffleSplit(n_splits=10, 
                  train_size=0.5, 
                  test_size=0.5)
for train_index, test_index in ss.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    clf.fit(X_train, y_train)
    print(clf.score(X_test, y_test))

# 層の割合を維持して分割
from sklearn.model_selection import StratifiedShuffleSplit
ss = StratifiedShuffleSplit(n_splits=1, 
                            train_size=0.95, 
                            test_size=0.05, 
                            random_state=0)
train_index, test_index = next(ss.split(X, y))
X_train, X_test = X[train_index], X[test_index]
y_train, y_test = y[train_index], y[test_index]

## cross validation

In [ ]:
from sklearn.model_selection import KFold
ss = KFold(n_splits=10, shuffle=True)
for train_index, test_index in ss.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

# 層の割合を維持して分割
from sklearn.model_selection import StratifiedKFold
ss = StratifiedKFold(n_splits=10, shuffle=True, random_state=2) # シャッフルしてから分割。
for train_index, test_index in ss.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

# scoreまで一気に。
from sklearn.model_selection import cross_val_score
ave_score = cross_val_score(clf, X, y, cv=10) # StratifiedKFold
# ave_scoreは、10個のスコアリスト。
print("{0:4.2f} +/- {1:4.2f} %".format(ave_score.mean() * 100, ave_score.std() * 100))

# leave one out
from sklearn.model_selection import LeaveOneOut
loocv = LeaveOneOut()
train_index, test_index = next(loocv.split(X, y))
# scoreまで一気に。
from sklearn.model_selection import cross_val_score
scores = cross_val_score(clf, X, y,cv=loocv) # LeaveOneOut

# leave p out
from sklearn.model_selection import LeavePOut
lpocv = LeavePOut(2)
scores = cross_val_score(clf, X, y,cv=lpocv)

# leave one-group out
from sklearn.model_selection import LeaveOneGroupOut
logocv = LeaveOneGroupOut()
group = np.array(list(range(50))*12)
group = np.sort(group[:y.size])
for train_index, test_index in logocv.split(X, y, group):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
from sklearn.model_selection import cross_val_score
scores = cross_val_score(clf,X, y,groups=group,cv=logocv) # LeaveOneGroupOut

# ハイパーパラメータの調整

 # 評価指標

## 正解率

In [1]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

## confusion matrix

行が真、列が予測。  
\begin{pmatrix}
True Negative & False Positive \\
False Negative & True Positive \\
\end{pmatrix}  
0と1では間違うことの重要性が違うので、混合行列を観る。  
どのラベルが苦手なのかもわかる。

In [ ]:
from sklearn.metrics import confusion_matrix
conf_mat = confusion_matrix(y_test, y_pred)
df = pd.DataFrame(conf_mat, 
                  columns=range(0,10), 
                  index=range(0,10))

In [ ]:
# 特徴が多い時は、画像で見る。
import matplotlib.pyplot as plt
%matplotlib inline
plt.gray()

plt.imshow(1- conf_mat / conf_mat.sum(axis=1), # 横方向に正規化
           interpolation='nearest') # 補間はしない

plt.yticks(range(20), newsgroups_train.target_names);
plt.xticks(range(20), newsgroups_train.target_names, rotation=90);

## classification_report

precision, recall, f1が一発で出る。  
averageは、support数で重みを付けた平均。  
表示するだけ。

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred, digits=4))

## f1_score

In [ ]:
from sklearn.metrics import f1_score
f1_score(y_test, y_pred, pos_label=0)
f1_score(y_test, y_pred, pos_label=1)

## fbeta_score

In [ ]:
from sklearn.metrics import fbeta_score
fbeta_score(y_test, y_pred, beta=1, pos_label=0), fbeta_score(y_test, y_pred, beta=1, pos_label=1)
# beta=1 が、f1_score

## precision_recall_fscore_support

precision, recall, f1が一発で出る。  
値を受け取れる。

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
precision_recall_fscore_support(y_test, y_pred, beta=1)

## ROC

In [ ]:
from sklearn.metrics import roc_curve, auc, average_precision_score, precision_recall_curve

test_score = clf.decision_function(X_test) # 2クラスは、各サンプルに対して、1つ定まる。
fpr, tpr, _ = roc_curve(y_test, test_score)
plt.plot(fpr, tpr, label="result")
print("result AUC = ", auc(fpr, tpr))

test_score = np.random.uniform(size=y_test.size)# もしまったくランダムなら
fpr, tpr, _ = roc_curve(y_test, test_score)
plt.plot(fpr, tpr, label="random / chance")
print("chance AUC = ", auc(fpr, tpr))

fpr, tpr, _ = roc_curve(y_test, y_test) # 完璧なら
plt.plot(fpr, tpr, label="perfect")
print("perfect AUC = ", auc(fpr, tpr))

plt.plot([0, 1], [0, 1], linestyle='--')
plt.legend(loc="best")
plt.xlim([-0.01, 1.01])
plt.ylim([0.0, 1.01])
plt.ylabel('True Positive Rate (recall)')
plt.xlabel('False Positive Rate (1-specificity)');

# 多クラス
test_score = clf.decision_function(X_test) 
# 多クラスは、各サンプルに対して、クラスの数分定まる。
# 10クラスならば、各サンプルについて、10成分の配列が定まる。
# 第0成分は、0を1としたときの距離。第1成分は、1を1としたときの距離。等。
for i in range(10):
    fpr, tpr, _ = roc_curve((y_test == i).astype(int), 
                            test_score[:,i])
    plt.plot(fpr, tpr, label="{0}, {1:.2f}".format(i, auc(fpr, tpr)))

plt.plot([0, 1], [0, 1], linestyle='--')
plt.legend(loc="best", title="class, AUC")
plt.xlim([-0.01, 1.01])
plt.ylim([0.0, 1.01])
plt.ylabel('True Positive Rate (recall)')
plt.xlabel('False Positive Rate (1-specificity)');

## PR

In [ ]:
from sklearn.metrics import roc_curve, auc, average_precision_score, precision_recall_curve
test_score = clf.decision_function(X_test)
precision, recall, _ = precision_recall_curve(y_test, test_score)
plt.plot(recall, precision, label="result")

test_score = np.random.uniform(size=y_test.size) # もしまったくランダムなら
precision, recall, _ = precision_recall_curve(y_test, test_score)
plt.plot(recall, precision, label="random")

precision, recall, _ = precision_recall_curve(y_test, y_test) # 完璧なら
plt.plot(recall, precision, label="perfect")

plt.legend(loc="best")
plt.xlim([-0.01, 1.01])
plt.ylim([0.0, 1.01])
plt.xlabel('Recall')
plt.ylabel('Precision')

# 多クラス
test_score = clf.decision_function(X_test) 
for i in range(10):
    precision, recall, _ = precision_recall_curve((y_test == i).astype(int), 
                            test_score[:,i])
    plt.plot(recall, precision, label="{0}".format(i))
             
plt.legend(loc="best")
plt.xlim([-0.01, 1.01])
plt.ylim([0.0, 1.01])
plt.xlabel('Recall')
plt.ylabel('Precision')


http://homepages.inf.ed.ac.uk/ckiw/postscript/ijcv_voc09.pdf for average precision and interpolation

PR図は、ギザギザになるので滑らかにする。  
recallの特定点でprecisionを補完して、平均をとる。

In [1]:
def calc_AP(precision, recall):
    precision_interp = np.maximum.accumulate(precision) # 0成分から右に見ていき、下がればそのままの値、上がれば上がった値にする。
    all_precision = np.interp(np.arange(0, 1.1, 0.1), recall[::-1], precision_interp[::-1]) # recallは降順なので、昇順に変える。
    AP = all_precision.mean()
    return AP

test_score = clf.decision_function(X_test)
precision, recall, _ = precision_recall_curve(y_test, test_score)
calc_AP(precision, recall)

# 多クラス
APs = []
for i in range(10):
    precision, recall, _ = precision_recall_curve((y_test == i).astype(int), test_score[:,i])
    APs.append( calc_AP(precision, recall) )
APs = np.array(APs)
mAP = APs.mean()
print(APs)
print("mAP = ", mAP)